# AI4ER Environmental Risk Practical Session: Image Analysis of Volcanic Rocks
- Friday 6th November, 2020 - online session - postponed to 13th.
- Lecturer: John Maclennan
- Practical Experts: Matt Ball and Norbert Toth.

## Purpose of this session
### Broader Context
Image analysis is a field with clear targets for machine learning approaches. Furthermore, many environmental applications use spectral imaging - the acquisition of a spectrum at each pixel. This is a language that will likely become familiar to you from the treatment of satellite data to map features on the global scale. At the other extreme of spatial scales, the examination of environmental materials by electron microscopy is evolving rapidly due to rapid advances in instrumentation the application of techniques developed in materials science to earth and environmental problems. 
### Key Skills
- Run Jupyter Notebook in Jupyter Lab from Binder
- Use of HyperSpy package for handling electron microscope data
- Use of Skimage package for image processing
- Application of Machine Learning techniques to Phase Identification
- Masking and Segmentation of phases
- Extraction of rock textural properties (Crystal Sizes and Shapes)
- Estimating magmatic timescales

## Table of Contents <a id='link1'></a>

<a href='#link10'> **1.0 What is a Phase Map of a Volcanic Rock?**</a>

<a href='#link20'> **2.0 Using Machine Learning to guide phase mapping**</a>

<a href='#link21'> **2.1 Load in Packages - Numpy, Scipy, Skimage, Hyperspy**</a>

<a href='#link22'> **2.2 Load and Plot EDS Data**</a>

<a href='#link23'> **2.3 Compositional information in Spectra**</a>

<a href='#link30'> **3.0 Machine Learning for Separating Phases**</a>

<a href='#link31'> **3.1 Identifying the number of phases**</a>

<a href='#link32'> **3.2 Defining the number of principal components**</a>

<a href='#link40'> **4.0 Mask extraction**</a>

<a href='#link50'> **5.0 Plotting a Phase Map**</a>

<a href='#link60'> **6.0 Extracting Textural Information from the Maps**</a>

<a href='#link61'> **6.1 Modal Analysis**</a>

<a href='#link62'> **6.2 Segmentation of Image into Crystals**</a>

<a href='#link62'> **6.3 Crystal Size Distributions**</a>

<a href='#link64'> **6.4 Aspect Ratios**</a>



This notebook contains a commented workflow on running machine learning on EDS data for phase separation and quantifying morphology.

Original written by Matt Ball, updated by John Maclennan and with new code from Norbert Toth.

## 1.0 What is a Phase Map of a Volcanic Rock? <a id='link10'></a>

Here is an example of a [phase map](https://academic.oup.com/view-large/figure/82008133/egu058f10p.png) of a rock from the active volcanic zones of Iceland.
The image is from [Neave et al., 2014](https://academic.oup.com/petrology/article/55/12/2311/1558945).
It was generated using a commerical system called QEMSCAN that is a piece of software that is linking to a scanning electron microcope (SEM).
You can see from the scale that we are looking at map of a 'thin section' prepared from the rock - you saw some of these in the lecture slides, as well as the SEM in the Department of Earth Sciences. The rocks are made of 'phases'. Each phase has a distinctive composition and set of thermodynamic properties which are determined by the crystalline (or glass) structure of the cooled lava. These phase names (e.g. olivine, plagioclase, clinopyroxene) are displayed on the legend of the figure along with the colours. In essence the QEMSCAN system is performing an advanced form of colour-by-numbers. The question is - how does it do this?

This raw data that goes into generation of these maps is acquired using a technique known as Energy Dispersive Spectroscopy (EDS). The SEM fires an electron beam at a focussed point on the sample surface, causing electrons to shift around energy levels and then to release X-rays - which are gathered by the EDS detectors. A spectrum is generated and the position of peaks can be attributed to characteristic energies that are associated with elements. The peak height is linked to the concentration of the element in the sample.

The QEMSCAN software converts the spectrum for each pixel into an estimate of the chemical composition of that point, and then uses a series of logical filters on the compositional data to match each pixel to a phase. For example, olivine is a mineral that contains This means that a large database of phases has to be accessed and the selection of possible phases is not guided by the compositional variations observed within the sample itself.

## 2.0 Using Machine Learning to guide phase mapping: Setup <a id='link20'></a>

An alternative approach with considerable promise in this field is to base the identification of phases directly on the available EDS data. In order to understand how we might be able to use this approach we are going to work through a set of activties in this notebook, starting from data input/output through to quantification of the rock textures.

The datasets are large and some of the ML processing is computationally expensive. For this practical we will therefore work with relatively small portions of the images, but will import results from larger images that have been calculated beforehand - Matt or Norbert might well say - "Here's one that I made earlier".

### 2.1 Load in Packages - Numpy, Scipy, Skimage, Hyperspy <a id='link21'></a>

In [34]:
import hyperspy.api as hs
hs.preferences.GUIs.enable_traitsui_gui = False
hs.preferences.save()
import numpy as np
from numpy import mean, std, median, sqrt, exp, log, delete

import matplotlib.pyplot as plt
import matplotlib as mpl
from cycler import cycler

import scipy
from scipy import ndimage as ndi
from scipy.stats import sem, t, norm, gaussian_kde, mstats, iqr
from scipy.optimize import curve_fit

import skimage as ski
from skimage.segmentation import random_walker
from skimage.feature import peak_local_max
from skimage import exposure
from skimage import measure
from skimage import morphology
from skimage import restoration
from skimage.filters import threshold_otsu
from skimage.color import rgb2gray

%matplotlib qt

### 2.2 Load and Plot EDS Data <a id='link22'></a>

Use Hyperspy to load in the EDS data in its bespoke .bcf format as output by the software that runs the detectors. You can look in the Hyperspy documentation to figure out what some of these commands are doing. 

In [2]:
#a = hs.load('your_filename.bcf')
a = hs.load('JM1_OL7_map1.bcf')
#https://drive.google.com/file/d/1yHQXNsE8Pls8lfTuLQTI9R46uqpY-Q0s/view?usp=sharing 
#gdrive_file_id = '1yHQXNsE8Pls8lfTuLQTI9R46uqpY-Q0s'
#a = hs.load(f'https://docs.google.com/uc?id={gdrive_file_id}&export=download')

a

[<Signal2D, title: Ch 0, dimensions: (|1820, 1295)>,
 <Signal2D, title: Ch 0, dimensions: (|2000, 2000)>,
 <EDSSEMSpectrum, title: EDX, dimensions: (1820, 1295|2048)>]

In [3]:
s = a[2]
s.axes_manager.set_signal_dimension(1)

In [4]:
#hs.preferences.gui()

In [5]:
s.plot()

Two new plot windows should have appeared. They does not look very impressive. One plot is a map and you can see the scale bar.
The bottom plot shows a spectrum, with a very small number of points. Can you figure out from the [Hyperspy manual](http://hyperspy.org/hyperspy-doc/current/user_guide/visualisation.html#multidimensional-spectral-data) what you have plotted here?

If you zoom in to the origin (look carefully at all the information on the plots) then you should be able to see a red line round the selected pixel. You can move this point (as long as pan and zoom are not selected) and examine the spectra of other individual pixels.

In [6]:
plt.close('all')

Try to play around with the Region of Interest commands in Hyperspy plotting. This should start to give you an even clearer feeling for what the low-quality spectrum - low number of counts - available at each point looks like.

In [7]:
roi = hs.roi.RectangularROI(left=120, right=460., top=300, bottom=560)
s.plot()
sc = roi.interactive(s)
sc.plot()

using interactive roi


In [8]:
plt.close('all')

removing widget


### 2.3 Compositional information in Spectra <a id='link23'></a>

Hyperspy contains a database of [elemental properties](http://hyperspy.org/hyperspy-doc/current/user_guide/eds.html#elemental-database). Add a code cell in below to print where you would expect to find peaks for Mg and Fe in the spectra. 

It is also possible to add labels to peaks found on the plotted spectra, as shown below. You might like to compare these with a high quality spectrum (lots of counts) for the mineral phase called olivine (e.g. in the [Mindat database](https://www.mindatnh.org/Forsterite%20Gallery.html)).

In [9]:
s.add_elements(['Si','Mg','Fe','Al','Ca']) # it is also possible to access detected elements from the bcf file from s.metadata.Sample.elements
s.plot(True)

In [11]:
plt.close('all')

### 2.4 Subset the loaded data <a id='link24'></a>

The data file provided is very large and too big for us to process in the Binder environment (2 Gb memory) or on typical desktops or laptops from 2019/20. It is therefore necessary to subset the data before carrying out computationally demanding work.

In [10]:
s_mini = s.data[0:200,500:800,:]
s_min = hs.signals.Signal1D(s_mini)
s_min.plot()

In [11]:
plt.close('all')

### Assign metadata

In [12]:
s_min.set_signal_type("EDS_SEM")
s_min.axes_manager[-1].name = 'E'
s_min.axes_manager['E'].units = 'keV'
s_min.axes_manager['E'].scale = 0.01
#Set the offset, the difference between the measured peak location and the true location, often an
#artificial zero peak is added to EDS data which can be used for this. This may need to be done a few
#times to correctly set this value
s_min.axes_manager['E'].offset = -0.0
#Set the beam energy to the beam energy the EDS data was collected at in keV
s_min.metadata.Acquisition_instrument.SEM.beam_energy = 20
s_min.plot()

In [13]:
plt.close('all')

### Saving and Loading in Hyperspy Format

When files are saved in Hyperspy format they can be rapidly reloaded as required. 



In [14]:
s.save('your_filename') # This is saving the full file (not the subset area) in hspy

Overwrite 'your_filename.hspy' (y/n)?
 y


In [15]:
s = hs.load('your_filename.hspy')

## 3.0 Machine Learning for Separating Phases <a id='link30'></a>

### 3.1 Identifying the number of phases <a id='link31'></a>

First we run a PCA (Principal Component Analysis) on the data to identify the number of significant principal components. The different phases will consist of both a loading map and corresponding spectrum. These can be cycled through using the arrow keys. Also plotted are two scree plots, which show the percentage of the data which can be explained by each principal component, one of which is plotted with a logarithmic y-axis, due to the large amount of data, this will be too slow to perform on the full dataset, so we will instead use the subset of data we created earlier, whilst the results of the machine learning on the full set can be loaded in and viewed.
The cell below produces a number of plots in windows - make sure that you can see them all and interact with them - using the slider window in particular.

In [16]:
s_min.change_dtype('float32')
s_min.decomposition(True, algorithm='SVD', output_dimension=20)
s_min.plot_decomposition_results()
s_min.plot_explained_variance_ratio(log=False)
s_min.plot_explained_variance_ratio(log=True)

Decomposition info:
  normalize_poissonian_noise=True
  algorithm=SVD
  output_dimension=20
  centre=None


<AxesSubplot:title={'center':'\nPCA Scree Plot'}, xlabel='Principal component index', ylabel='Proportion of variance'>

It is worth noting that the results of this PCA contain both negative loading intensities as well as negative spectra, neither of which is physically meaningful. This first step is required to identify the number of significant phases which later machine learning can pull apart. How many phases do you think are being picked up by this PCA approach? How can we tell whether the recovered principal components are likely to correspond to natural features of the rock sample?

In [17]:
plt.close('all')

Again we can save the results of this analysis

In [18]:
s.learning_results.save('your_filename_PCA')

Here's one I made earlier! Now we can load in the PCA results on the whole dataset to compare to our subsection. Alternatively we can load in results of any previous PCA

In [19]:
s.learning_results.load('AI4ER_JM1_OL7_PCA.npz')

In [20]:
s.plot_decomposition_results()
s.plot_explained_variance_ratio(log=False)
s.plot_explained_variance_ratio(log=True)

<AxesSubplot:title={'center':'EDX\nPCA Scree Plot'}, xlabel='Principal component index', ylabel='Proportion of variance'>

Again, how many of the PCs do you think might be picking up aspects of the rock sample - rather than analytical noise or artifacts? The answer here might be different to the previous case - because here we are looking at the results from the full area that was scanned, not a subset. 

In [21]:
plt.close('all')

### 3.2 Defining the number of principal components <a id='link32'></a>

The number of principal components can be defined either by the number which is below a critical value of variance, by the shape of the scree plot, by the form of the associated spectrum or by the spatial pattern defined in the images. Pick a number - perhaps discuss the best strategy with a demonstrator if you are unsure.

In [22]:
PC = 'number of principal components' # Replace string with number
PC = 3

We now use this number of principal components to run a non-negative matrix factorisation, a clustering algorithm which produces only positive loadings and spectra, removing the issue of non physically meaningful results. This makes use of machine learning algorithms through scikit-learn. Again we will perform this only on the subset of the data and load in a previously collected result for the full dataset.

In [23]:
s_min.decomposition(True, algorithm='NMF', output_dimension=PC)
s_min.plot_decomposition_results()

Decomposition info:
  normalize_poissonian_noise=True
  algorithm=NMF
  output_dimension=3
  centre=None
scikit-learn estimator:
NMF(n_components=3)


This should be starting to look quite good - use the slider window to examine the maps and spectra associated with the factors obtained.  Hopefully you can see crystal shapes on the maps and note that the spectra look physically plausible - in that at least they all have positive count values. Another remarkable feature is how smooth and sharp the spectra associated with each factor are - certainly in comparison to the noisy low-count spectra from each individual point. This is the power of looked for correlated signals across thousands of pixels. 

In [24]:
plt.close('all')

Again we can save the results of this analysis

In [25]:
s.learning_results.save('your_filename_NMF')

And again here's another pre-NMF-ed set of results. We could alternatively load in other previously collected results

In [26]:
s.learning_results.load('AI4ER_JM1_OL7_NMF.npz')

Now, plot these results up and attempt to label the spectra for the position of the elements.

In [31]:
s.plot_decomposition_results()

In the next plot, you can move between the spectra of the factors by clicking on the red line in the slider and moving it vertically. 

In [28]:
snspec = s.get_decomposition_factors()
snspec.add_elements(['Si','Mg','Fe','Al','Ca','Cr'])
snspec.plot(True)

Which phases do the different factors correspond to? Use the [Mindat](https://www.mindat.org/) database to check on mineral compositions.
In this type of rock, which is called a basalt, the common mineral phases are olivine, clinopyroxene, plagioclase, magnetite and a Cr-bearing spinel. **Be careful - the peaks at less than about 1 keV show overlap here which makes it difficult for the peak picking alhorithm to always correctly label a peak with the correct element. Also - one of the tallest peaks may be labelled with the element right at the top of the plot.**

The basaltic liquid also quenches to a glass when it cools rapidly upon eruption. 
You can find some likely glass compositions (as well as the answers to the questions above) in [Larsen and Pedersen, 2000](https://academic.oup.com/petrology/article/41/7/1071/1457495#99082445) - glass compositions in Table 3.

In [37]:
plt.close('all')

## 4.0 Mask extraction <a id='link40'></a>

The above NMF factors are not yet in their most useful form - they need to be converted to 'binary images': images of only 1s and 0s. With such 'masks' we can start to extract information about the seprated phases. This can be done manually by setting a suitable threshold below which all pixels are 0 and 1 above or automatically by using a suitable algorithm - eg. [Otsu's method here](https://en.wikipedia.org/wiki/Otsu%27s_method) or [here](https://scikit-image.org/docs/0.7.0/api/skimage.filter.thresholding.html).

In [30]:
phase_images = []
phases = [0,1,2,3]
components = s.get_decomposition_loadings()

for i in phases:
    #.compute() is required below due to lazy loading of data
    image = components.inav[i].data.copy()
    image = image/image.max() #normalize loading values
    
    phase_images.append(image)
    
    #creating bins from 0.01 to 1 with width 0.01.
    bins = [0.01*(n+1) for n in range(99)]
    
    fig, ax = plt.subplots()
    ax.hist(image.flatten(), bins)
    ax.set_xlabel("Value")
    ax.set_ylabel("Frequency")

Check that you understand what these four plots mean. How might you use them to create masks? What are the likely causes of spread on the histograms? It may be helpful to go back and look at the maps of the loadings from the end of the last section.

The cell below shows some manual choices of value for the binary thresholding. You might want to test what happens if you use different values or explore the Otsu method referred to above for these choices. 

In [32]:
thresh = [0.26,0.3,0.36,0.18]
post_thresh = []

for n in range(len(phase_images)):
    img = phase_images[n].copy()
    
    img[img<thresh[n]] = 0
    img[img>thresh[n]] = 1
    
    post_thresh.append(img)
    
    fig, ax = plt.subplots()
    ax.imshow(img)

While this is starting to look promising, you might also be able to spot some artifacts on these images. Can you describe these? How do you think that you might deal with these artifacts?

Make sure that you have a clear idea in your mind about what the identity of each phase is (e.g. glass, plagioclase, olivine). 

One possible piece of further processing of the images to handle noise in the data can be carried out using 'morphological transformations' (https://scikit-image.org/docs/dev/api/skimage.morphology.html) to get rid of noise present due to having phases of similar compositions - one of the limitations of using EDS only. 'Area_opening' operation gets rid of objects lower than a given size and so it is used to remove single pixels or very small clusters of pixels which can be attributed to noise.

In [35]:
min_size = 10 #adjust this value or make into an array to give each mask an individual value
phase_mask = []

for n in range(len(post_thresh)):
    #if min_size is an array use min_size[n] below
    mask = morphology.area_opening(post_thresh[n], min_size)
    mask = morphology.area_closing(mask, min_size)
    phase_mask.append(mask)
    
    fig, ax = plt.subplots()
    ax.imshow(mask)

If you have any good ideas about how to eliminate other artifacts in these results, please do go ahead and writed some cells in your notebook to explore your thinking. For example, if you go all theway back to the original PCA results for the whole region, you might see a potential source of problems. 

## 5.0 Plotting a Phase Map <a id='link50'></a>

It is possible to combine these images into a single 'phase map'. 

In [38]:
#create blank image - all zeroes
phase_map = np.zeros((len(phase_mask[0]), len(phase_mask[0][0])))
abundance = []

for i in range(len(phase_mask)):
    coords = np.nonzero(phase_mask[i])
    abundance.append(len(coords[0]))
    for n in range(len(coords[0])):
        phase_map[coords[0][n]][coords[1][n]] = i+1

#should show the phase map in grayscale
plt.imshow(phase_map)

You can also produce a fancy version of this image, labelled up with your identified phases by editing the 'phase_names' line in the cell and your own colour scheme.

In [45]:
#replace with RGB codes for the desired colours
# index 0 is unclassified and the rest have to be in order of the phases in the above images from loadings
colour = ['black', 'palegoldenrod', 'olivedrab', 'darkcyan','purple' ]
cMap = mpl.colors.ListedColormap(colour)

#create plot
fig, ax = plt.subplots(figsize = (9,5))
plot = plt.imshow(phase_map, cmap=cMap, interpolation = None)

#get values present in above image
values = np.unique(phase_map.ravel())

# get the colors of the values, according to the colormap used by imshow
colors = [ plot.cmap(plot.norm(value)) for value in values]

# Name each colour and create a patch (proxy artist) for every color 
phase_names = ["Unclassified", "Phase1", "Phase2", "Phase3", "Phase4"]
patches = [ mpl.patches.Patch(color=colors[i], label=phase_names[i] ) for i in range(len(values)) ]

# put those patched as legend-handles into the legend
plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0. )

#set tick marks here, including how many to show
x_ticks = np.linspace(0,len(phase_map[0]),7)
y_ticks = np.linspace(0,len(phase_map),5)

# What those pixel locations correspond to in data coordinates.
# Also set the float format here
x_ticklabels = ["{:6.2f}".format(i) for i in (x_ticks*s.axes_manager[0].scale)/1000]
y_ticklabels = ["{:6.2f}".format(i) for i in (y_ticks*s.axes_manager[0].scale)/1000]

#apply the above changes to the plot
ax.set_xticks(x_ticks)
ax.set_xticklabels(x_ticklabels)
ax.set_yticks(y_ticks)
ax.set_yticklabels(y_ticklabels)

plt.title("Labelled phase map - axis values are shown in mm.", fontsize = 11)
plt.show()

fig.tight_layout()

Once you are happy with the plot, you can save it to a file.

In [46]:
#to save above plot:
fig.savefig("Example_labelled_map.png", dpi = 600)

## 6.0 Extracting Textural Information from the Maps <a id='link60'></a>

Rock textures can be used to understand the physical processes that occurred in a magmatic system in association with eruption.

### 6.1 Modal Analysis <a id='link61'></a>

This simple chacacterisation is used to describe the volumetric proportions of the phases in a sample.




In [48]:
names = phase_names[1:].copy()

#measure phase proportions by area here
total_pix = sum(abundance)

phase_proportions = []
for i in range(len(abundance)):
    phase_proportions.append(abundance[i]/total_pix)
    print("Modal " + names[i] + " proportion: " + str(phase_proportions[i]))
    
fig, ax = plt.subplots()

x = range(len(abundance))
ax.bar(x, phase_proportions,tick_label = names)
ax.set_ylabel("Modal abundance by area")
fig.suptitle("Modal phase proportions")

Modal Phase1 proportion: 0.5982155830081246
Modal Phase2 proportion: 0.3547650548965913
Modal Phase3 proportion: 0.04522879135192792
Modal Phase4 proportion: 0.001790570743356085


Text(0.5, 0.98, 'Modal phase proportions')

### 6.2 Segmentation of Image into Crystals <a id='link62'></a>

Using standard Data Analysis libraries - suck as skimage, it is possible to rapidly quantify crystal sizes and shapes from binary phase masks generated above. See the 'measure' module within skimage for more info - https://scikit-image.org/docs/dev/api/skimage.measure.html.

In this example the Plagioclase phase mask will be used to show how this may be done. The first step is to actually determine where the different crystals are within the phase mask and label them. A simple approach is to label by connectivity of pixels, however it assumes separate crystals are not touching and clusters can lead to significant errors. An image processing approach is to use marker based watershed segmentation to separate clusters; it can work well, but requires some manual tweaking. The algorithm is based on segmentation of touching circles - it doesn't perform as well with non circular objects. The smartest approach may be to use EBSD data to segment clusters based on contrast from crystallographic orientation.

Experiment with the area_closing function to see how effectively it fills in the gaps in the crystals. 

In [52]:
#filled objects are best to perform watershed as inclusions are treated as 'background' in binary mask so it may
#be worth filling in the crystals using morphological transformations to improve segmentation quality

plag_mask = phase_mask[2].copy()
plag_mask = morphology.area_closing(plag_mask, 64)

fig, ax = plt.subplots(figsize = (12,24))
ax.imshow(plag_mask)

Next, perform the segmentation with the watershed algorithm. Again, try to experiment with to see how effectively you can segment the objects. Each object is given a different colour here - you will be able to spot some large crystals that have been incorrectly segmented, for example. 

In [55]:
ws_mask = plag_mask.copy()

distance = ndi.distance_transform_edt(ws_mask)

#tune footprint to get wanted/optimum segmentation of touching objects?
#large objects may get oversegmented whilst smaller ones won't segment at all - it's impossible to get it perfect
local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((25,25)), labels=ws_mask)

markers = ski.morphology.label(local_maxi, connectivity = 2)
labels_ws = ski.segmentation.watershed(-distance, markers, mask=ws_mask)

fig, ax = plt.subplots(figsize = (12,24))
ax.imshow(labels_ws, cmap = 'twilight')

### 6.3 Crystal Size Distributions <a id='link63'></a>

The plagioclase crystals tend to be elongate and it is therefore useful to define the size (a length) as the square root of the area as counted from the number of pixels in each segmented object. We can then start by plotting a fequency-size histogram. 

In [64]:
#get spatial resolution of individual pixels; the 0th axis is a spatial dimension so can be used here
#if it wasn't clear it's a spatial dimension it can be modified to: 
scale = s.axes_manager.navigation_axes[0].scale
#scale = s.axes_manager[0].scale


plag = labels_ws.copy()
properties = measure.regionprops(labels_ws)
ROI_area = scale**2 * len(plag)*len(plag[0])

#in the approximate CSD size is measure in Area^{0.5}
size = [np.sqrt(cryst.area)*scale for cryst in properties]

#creating bins from 0.01 to 1 with width 0.01.
bins = [1.0*(n+1) for n in range(99)]
fig, ax = plt.subplots()
ax.hist(size, bins)
ax.set_xlabel("Area^{0.5}")
ax.set_ylabel("Frequency")


Text(0, 0.5, 'Frequency')

Try now to consider how the limitations of the data collection and the processing steps used in this practical might influence the appearance of this histogram. In other words, is this likely to be a good approximation to the natural underlying distribution of crystal sizes in the rock sample?

In the recent paper by (Neave et al., 2018)[https://www.degruyter.com/view/journals/ammin/102/10/article-p2007.xml] the current favoured method for characterisinhg crystal size distributions is developed, which effectively provides the gradient on the slope of a cumulative distribution of size (normalised to unit area). A number of functions are defined below to perform these calculations.

In [65]:
def geo_factor(min_size, max_size, n_bins):
    """
    Calculate the geometric factor required to establish bins.
    
    Input
    --------------------------------------
    min_size - lower bound of binning process
    max_size - upper bound of binning
    n_bins - number of bins generated 
    
    Returns
    --------------------------------------
    factor - the geometric factor required for the binning process
    correction - log (base factor) of lower bound of the binning process
    """
    ratio = max_size/min_size
    factor = ratio**(1/n_bins)
    correction = np.log(min_size)/np.log(factor)
    return factor, correction
    
    
def gen_geo_bins(min_size, max_size, n_bins):
    """
    Generate geometric bins required.
    
    Input
    ----------------------------------
    min_size - lower bound of binning process
    max_size - upper bound of binning
    n_bins - number of bins generated   
    
    Returns
    -----------------------------------------------------
    bins - lower bound for each bin
    bw - bin widths
    x_grid - middle values for the bins to be used for plot
    """
    factor, correction = geo_factor(min_size, max_size, n_bins)
    
    bins = [factor**(i+correction) for i in range(n_bins)]
    bw = [(factor**(i+1 + correction) - factor**(i + correction)) for i in range(n_bins)]
    x_grid = [(factor**(i+1 + correction) + factor**(i + correction))/2 for i in range(n_bins)]
    
    return bins, bw, x_grid

def gen_csd_plot(size, min_size, max_size, n_bins, area):
    """
    Create approximate CSD plot (Neave et. al. 2017) using geometric binning.
    
    Inputs
    ----------------------------------------------
    size - array of crystal sizes
    min_size - lower bound of binning process
    max_size - upper bound of binning
    n_bins - number of bins generated 
    area - area of ROI considered
    
    Returns
    -----------------------------------------------
    fig - matplotlib figure object containing the CSD plot
    ax - matplotlib axis object for fig above
    """
    
    #run functions to generate bins and all required parameters
    factor, correction = geo_factor(min_size, max_size, n_bins)
    bins, bw, x_grid = gen_geo_bins(min_size, max_size, n_bins)
        
    n_cryst = np.zeros(n_bins)

    #apply binning to data
    for i in range(len(size)):
        if size[i] >= max_size:
            pass
        elif size[i] < min_size:
            pass
        else:
            index = int((np.log(size[i])/np.log(factor))-correction)
            n_cryst[index] += 1
    
    #generate y axis data
    n_cryst_div_bw = [(n_cryst[i]/bw[i])/area for i in range(n_bins)]
    ln_cryst_div_bw = np.log(n_cryst_div_bw)
    
    #create matplotlib plot that is then returned
    fig, ax = plt.subplots()
    
    ax.plot(x_grid, ln_cryst_div_bw, '.', markersize = 11)
    ax.set_xlabel(r"Area$^{0.5}$ ($\mu$m)")
    ax.set_ylabel("ln(N / bw)")
    
    fig.show()
    
    return fig, ax

The following cell generates a plot for the plagioclase areas in the original sample. The y-axis is the number of crystals within a certain size bin within in the area, so has units of mm$^{-3}$. You can experiment with the range and the bin-width. 

In [67]:
#the function used to generate plot uses geometric binning
#it requires the following input: size, min_size, max_size - minimum and maximum to be used for binning
#n_bins - number of bins used
#area - area of entire ROI 
distribution, ax = gen_csd_plot(size, 5,75,12,ROI_area)

<ipython-input-65-cc3fdeaf8b97>:82: RuntimeWarning: divide by zero encountered in log
  ln_cryst_div_bw = np.log(n_cryst_div_bw)


It is possible to use such plots to estimate the timescales of crystal growth, such as that shown on Figure 4 of [Cashman and Marsh](https://link.springer.com/content/pdf/10.1007/BF00375363.pdf). Try to estimate $G\tau$ from the slope of your plot (it would be possibel to write a few lines of code to quantify this). You can see in Table 2 of this paper that some values for plagioclase growth rates, $G$ are provided. Use this information to estimate the timescale of crystal growth. 

### 6.4 Aspect Ratios <a id='link64'></a>

The aspect ratio is the length over width of the crystal. We know from crystallisation experiments on magma samples that high values tend to indicate rapid crystal growth and are associated with short timescales. The paper of Holness et al. 

In [71]:
plt.close('all')

In [72]:
aspect_ratio = [cryst.major_axis_length/cryst.minor_axis_length for cryst in properties]
#can apply correction proposed by Neave et. al. (2017) to aspect ratios
#aspect_ratio_corr = [(1.150*item - 0.195) for item in aspect_ratio]

plt.hist(aspect_ratio)
plt.xlabel("Aspect Ratio")
plt.ylabel("Frequency")

fig, ax = plt.subplots()
ax.plot(size, aspect_ratio, '.')
ax.set_xscale('log')
ax.set_xlabel(r"Area$^{0.5}$ ($\mu m$)")
ax.set_ylabel("Aspect Ratio")

Text(0, 0.5, 'Aspect Ratio')

You can try to use the calibration of plagioclase aspect ratios by [Holness et al., 2015](https://link.springer.com/content/pdf/10.1007/s00410-014-1076-5.pdf), presented on p1076 of that paper, to try to estimate crystallisation times. 

Try to compare your estimates of crystallisation times from the CSDs and the aspect ratios. They might be *extremely* different to each other. Try to consider why these estimates might differ. What could you do to improve the image processing to remove that as a cause of inconsistency between the approaches? What other assumptions in the approach might also leas to inconsistencies in timescales of estimates? 

## 7.0 Extra: Application to another dataset <a id='link70'></a>

In the introduction to this exercise we considered the merits of the QEMSCAN approach. This software is highly effective at mapping large areas of thin sections rapidly and generating phase maps. A .png file generated by filtering the rough compositional map from a QEMSCAN image is provided in the Moodle folder. This has been filtered to only show positive values in regions where the Mg content is above a threshold (i.e. is olivine) and show a strength of green colour defined by the iron content. If you have the time and the inclination, see if you can read this file into HyperSpy and try to extract some quantitative information about its textures.